In [ ]:
%%capture
!python -m pip install --upgrade pip -q
!pip install transformers  -q -U
!pip install peft  -q -U
!pip install accelerate  -q -U
!pip install flash  -q -U
!pip install  datasets -q -U
!pip install  scipy -q -U
!pip install  trl -q -U
!pip install  hf_transfer -q -U
!pip install  huggingface_hub -q -U
!pip install  wandb -q -U
!pip install -U bitsandbytes

In [ ]:
!transformers-cli env

2024-07-19 20:11:08.872642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 20:11:08.872697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 20:11:08.874737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 20:11:08.886524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 20:11:11.197932: W tensorflow/comp

# Load the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch

cache_dir = ''
model_id = ""

### bits and byte

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # if newer gpu: bfloat16
)

### main model

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #config=config,
    # quantization_config=bnb_config,

    # rope_scaling={"type":'dynamic', "factor": 2.0}, # roPE scaling: https://www.hopsworks.ai/dictionary/rope-scaling and https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/preparing_model

    #https://huggingface.co/docs/accelerate/v0.25.0/en/concept_guides/big_model_inference
    # device_map='auto', # It’s fully possible to create your own device map for the layers to use as well, specifying the GPU device to use (a number), "cpu", or "disk" and pass this in:
    device_map = {"": 0},
    torch_dtype=torch.float16, # if newer gpu: bfloat16

    # https://huggingface.co/docs/text-generation-inference/en/conceptual/flash_attention
    # attn_implementation="flash_attention_2", # Works with llama model

    cache_dir = cache_dir
)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

# Checks

In [ ]:
for n, p in model.named_parameters():
  if p.device.type == "meta":
    print(f"{n} is on meta")

In [ ]:
print(tokenizer.bos_token)
print(tokenizer.eos_token)
print(tokenizer.pad_token)

<s>
</s>
</s>


In [ ]:
print(model.config.eos_token_id)

2


In [ ]:
# Check the integrity of the model and tokenizer after modifying the tokenizer

# Get the vocabulary sizes of the tokenizer
tokenizer_vocab_size = len(tokenizer)
model_embedding_size = model.get_input_embeddings().weight.shape[0]

# Compare the vocabulary size
if tokenizer_vocab_size == model_embedding_size:
  print("The tokenizer vocabulary size matches the model embedding size.")
else:
  print("The tokenizer vocabulary size does not match the model embedding size.")

# Print the size for verification
print(f"Tokenizer vocabulary size: {tokenizer_vocab_size}")
print(f"Model embedding size: {model_embedding_size}")

The tokenizer vocabulary size matches the model embedding size.
Tokenizer vocabulary size: 32000
Model embedding size: 32000


In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

# LoRA

In [ ]:
from peft import prepare_model_for_kbit_training

# look at gradient checkpointing and gradient accumulation on https://huggingface.co/docs/transformers/v4.18.0/en/performance
model.gradient_checkpointing_enable() # to save some memory in VRAM in turn for little slow training

# model = prepare_model_for_kbit_training(model) # for quantization, must be uncommented.

from peft import LoraConfig, get_peft_model

# Understaing Lora parameters: https://medium.com/@drishtisharma96505/comparative-analysis-of-lora-parameters-on-llama-2-with-flash-attention-574b913295d4
peft_config = LoraConfig( #matching the Llama recipe
                         r = 128,
                          lora_alpha = 32,
                          target_modules="all-linear",
                          lora_dropout = 0.1,
                          bias = "none",

                          )

model = get_peft_model(model, peft_config) #move to a peft model

In [ ]:
model.print_trainable_parameters()

trainable params: 100,925,440 || all params: 1,200,973,824 || trainable%: 8.4036


In [ ]:
print("Special tokens map:", tokenizer.special_tokens_map)
print( "All special tokens:", tokenizer.all_special_tokens)

Special tokens map: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}
All special tokens: ['<s>', '</s>', '<unk>']


In [ ]:
print(tokenizer.padding_side)

right


In [ ]:
tokenizer.padding_side = 'right'


LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.42.4",
  "use_cache": true,
  "vocab_size": 32000
}



In [ ]:
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 2048,
  "pad_token_id": 0
}



# Load the Dataset

In [ ]:
import json
from datasets import load_dataset, concatenate_datasets, DatasetDict

In [ ]:
dataset_0 = load_dataset("Telugu-LLM-Labs/nepali_alpaca_yahma_cleaned_filtered", split="train")
# dataset_1 = load_dataset("caspro/Nepali_News_Dataset", split="train")
dataset_2 = load_dataset("wisewizer/nepali-news", split="train")

Generating train split:   0%|          | 0/5808 [00:00<?, ? examples/s]

In [ ]:
dataset_3 = load_dataset("CohereForAI/aya_collection_language_split", "nepali")

Generating train split:   0%|          | 0/3576367 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/272872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/274905 [00:00<?, ? examples/s]

In [ ]:
dataset_from_pandas = dataset_3["train"].to_pandas()

In [ ]:
dataset_from_pandas.iloc[3576362]["inputs"]

'3/5 बाट 1/4 घटाउनुहोस् र परिणामलाई सरलीकृत अंशको रूपमा व्यक्त गर्नुहोस्।\n'

In [ ]:
alpaca_prompt = """Below is an instruction in Npelai that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["nepali_instruction"]
    inputs       = examples["nepali_input"]
    outputs      = examples["nepalioutput"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_0 = dataset_0.map(formatting_prompts_func, batched = True,)

In [ ]:
import random

dataset_1 = dataset_1.rename_column("text", "news")
alpaca_prompt = """Below is an instruction in Nepali that describes a task, paired with an input in Nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    titles     = examples["title"]
    all_summarized_news      = examples["summary"]
    all_news = examples["news"]

    texts = []
    for title, summarized_news, news  in zip(titles, all_summarized_news, all_news ):
        random_number = random.randint(0, 2)

        if random_number == 0:
          instruction = "यहाँ समाचार शीर्षक र संक्षेपित समाचार दिइएको छ। तिनीहरूबाट पूर्ण समाचार लेख लेख्नुहोस्।"
          input = f"शीर्षक: {title}, संक्षेपित समाचार: {summarized_news}"
          # Must add EOS_TOKEN, otherwise your generation will go on forever!
          text = alpaca_prompt.format(instruction, input, news) + EOS_TOKEN
        elif random_number == 1:
          instruction = "शीर्षक सहित दिइएको समाचार लेखको सारांश लेख्नुहोस्"
          input = f"शीर्षक: {title}, समाचार: {news}"
          text = alpaca_prompt.format(instruction, input, summarized_news) + EOS_TOKEN
        else:
          instruction = "दिइएको समाचार र सारांशबाट समाचार शीर्षक लेख्नुहोस्"
          input = f"समाचार: {news}, संक्षेपित समाचार: {summarized_news}"
          text = alpaca_prompt.format(instruction, input, title) + EOS_TOKEN

        texts.append(text)
    return { "text" : texts, }
pass


dataset_1 = dataset_1.map(formatting_prompts_func, batched = True,)

In [ ]:
import random

dataset_2 = dataset_2.rename_column("text", "news")
alpaca_prompt = """Below is an instruction in Nepali that describes a task, paired with an input in Nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    titles     = examples["title"]
    all_summarized_news      = examples["summary"]
    all_news = examples["news"]

    texts = []
    for title, summarized_news, news  in zip(titles, all_summarized_news, all_news ):
        random_number = random.randint(0, 2)

        if random_number == 0:
          instruction = "यहाँ समाचार शीर्षक र संक्षेपित समाचार दिइएको छ। तिनीहरूबाट पूर्ण समाचार लेख लेख्नुहोस्।"
          input = f"शीर्षक: {title}, संक्षेपित समाचार: {summarized_news}"
          # Must add EOS_TOKEN, otherwise your generation will go on forever!
          text = alpaca_prompt.format(instruction, input, news) + EOS_TOKEN
        elif random_number == 1:
          instruction = "शीर्षक सहित दिइएको समाचार लेखको सारांश लेख्नुहोस्"
          input = f"शीर्षक: {title}, समाचार: {news}"
          text = alpaca_prompt.format(instruction, input, summarized_news) + EOS_TOKEN
        else:
          instruction = "दिइएको समाचार र सारांशबाट समाचार शीर्षक लेख्नुहोस्"
          input = f"समाचार: {news}, संक्षेपित समाचार: {summarized_news}"
          text = alpaca_prompt.format(instruction, input, title) + EOS_TOKEN

        texts.append(text)
    return { "text" : texts, }
pass


dataset_2 = dataset_2.map(formatting_prompts_func, batched = True,)

# Train

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_datasets,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio=0.03,
        num_of_train_epochs = 3,
        # max_steps = 60,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "aya_1",
        report_to = "wandb",

    ),
)